<a href="https://colab.research.google.com/github/wangdaming155/dive-into-deep-learning2/blob/main/%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E5%AD%A6%E4%B9%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/d2l-zh')

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
X,net(X)#对应2*10，输入和输出

(tensor([[0.0014, 0.7160, 0.8261, 0.5242, 0.8588, 0.9524, 0.9499, 0.9801, 0.5825,
          0.6764, 0.4359, 0.5924, 0.8801, 0.8499, 0.6298, 0.1967, 0.1377, 0.9252,
          0.5817, 0.9889],
         [0.4365, 0.4024, 0.3789, 0.6921, 0.0363, 0.3134, 0.4161, 0.6376, 0.9560,
          0.3011, 0.8306, 0.1832, 0.0932, 0.5332, 0.3969, 0.1358, 0.8994, 0.1098,
          0.4370, 0.0728]]),
 tensor([[-0.1007, -0.1932,  0.3082, -0.3810,  0.1796, -0.1238,  0.0575,  0.0595,
           0.0034, -0.1783],
         [-0.0370, -0.1582,  0.1128, -0.1486,  0.0886, -0.1812, -0.0686,  0.1048,
          -0.0193, -0.1929]], grad_fn=<AddmmBackward0>))

## 每个块要具备的功能
将输入数据作为前向传播函数的参数
通过前向传播函数生出输出，输出可能和输入不同
计算输出关于输入的梯度，通过反向传播函数进行访问
存储和访问钱箱传播计算所需的参数
根据需要初始化参数

In [ ]:
# 实例化module
class MLP(nn.Module):
  # 用模型参数声明层，这里我们声明两个全连接层
  def __init__(self):
    #调用MLP的父类Module的构造函数进行必要的初始化
    #在类实例化时，可以指定其他的函数参数，例如模型参数的params
    super().__init__()
    self.hidden = nn.Linear(20,256) #隐藏层
    self.out = nn.Linear(256,10)#输出层

  #定义模型的前向传播，如何由输入到输出
  def forward(self,X):
    #注意，Relu函数的版本是根据nn中的funtional定义的
    return self.out(F.relu(self.hidden(X)))


In [ ]:
net = MLP()
net(X)


tensor([[ 0.0086, -0.0182,  0.1390,  0.1992,  0.1305,  0.1471,  0.0576, -0.1140,
          0.2060,  0.0379],
        [ 0.1007,  0.0260,  0.0512,  0.0959,  0.0986,  0.1613, -0.0030,  0.0878,
          0.0933,  0.0316]], grad_fn=<AddmmBackward0>)

## 顺序块
1.将块逐个追加到函数之中
2.前向传播函数，将输出按照追加块的顺序传递给块组成链条

In [ ]:
class MySequential(nn.Module):
  def __init__(self,*args):
    super().__init__()
    for idx,module in enumerate(args):
      #module为Module子类的一个实例，我们把它保存为成员
      #为有序字典
      self._modules[str(idx)] = module
  
  def forward(self,X):
    # orderdict guarantees that members will traversted in the order
    # they were added
    for block in self._modules.values():
      X = block(X)
    return X

In [ ]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.1153,  0.2893, -0.3591,  0.0572, -0.0590,  0.5113,  0.2822, -0.1310,
          0.1553,  0.0047],
        [-0.0368,  0.0482, -0.3109,  0.0548, -0.0725,  0.4493,  0.1925, -0.1859,
         -0.0244,  0.0100]], grad_fn=<AddmmBackward0>)

In [ ]:
"""executing code in the forward propagation function"""

'executing code in the forward propagation function'

In [ ]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the created constant parameters, as well as the `relu` and `mm`
        # functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [ ]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0859, grad_fn=<SumBackward0>)

In [ ]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1336, grad_fn=<SumBackward0>)

### parameter management

In [ ]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size= (2,4))
net(X)

tensor([[0.1759],
        [0.0683]], grad_fn=<AddmmBackward0>)

In [ ]:
"""Parameter Access"""

'Parameter Access'

In [ ]:
print(net[2].state_dict())
print(net[1].state_dict())
print(net[0].state_dict())

OrderedDict([('weight', tensor([[ 0.1436,  0.0105, -0.3266, -0.0712,  0.1226, -0.0725,  0.2711, -0.0759]])), ('bias', tensor([0.2219]))])
OrderedDict()
OrderedDict([('weight', tensor([[-0.3102,  0.1792, -0.3564,  0.0435],
        [ 0.1872, -0.0887, -0.1102, -0.0954],
        [ 0.2366,  0.2643,  0.1830,  0.1975],
        [ 0.2844,  0.3651, -0.3947,  0.2343],
        [-0.1519, -0.2913,  0.1795, -0.4846],
        [ 0.3810, -0.3746,  0.1543,  0.3679],
        [-0.3454, -0.0303,  0.3015,  0.0682],
        [ 0.2433,  0.4920, -0.1307,  0.4299]])), ('bias', tensor([-0.2430,  0.4449, -0.0117,  0.2131, -0.3819, -0.3970,  0.1603, -0.2451]))])


In [ ]:
"""targeted parameters"""
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2219], requires_grad=True)
tensor([0.2219])


In [ ]:
net[2].weight.grad == None  ### there is grad and data

True

In [ ]:
"All Parameters at Once"
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])  #there isn't 2 in relu

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [ ]:
net.state_dict()['2.bias'].data

tensor([0.2219])

In [ ]:
"""collecting parameter from  nested blocks"""

'collecting parameter from  nested blocks'

In [ ]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # nested here
        net.add_module(f'block{i}',block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.5052],
        [-0.5053]], grad_fn=<AddmmBackward0>)

In [ ]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [ ]:
rgnet[0][1][0].bias.data

tensor([-0.2443,  0.2802, -0.2391,  0.1032, -0.2388,  0.4226,  0.2559, -0.3467])

把数据打印出来，按照嵌套结构将数据进行访问

In [ ]:
"""Parameter initialization"""

'Parameter initialization'

In [ ]:
#builit in initialization
def init_normal(m):
  if type(m)==nn.Linear:
    nn.init.normal_(m.weight,mean=0,std=0.01)# 正态分布；constant 常数
    nn.init.zeros_(m.bias)
  
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([0.0072, 0.0049, 0.0097, 0.0142]), tensor(0.))

In [ ]:
def init_constant(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight,1)
    nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]  #寻找网络的参数，首选寻找到网络的位置

(tensor([1., 1., 1., 1.]), tensor(0.))

In [ ]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)
#先架构出函数，来负责给初始化，然后利用net[].apply()分块初始化


tensor([ 0.4186, -0.5413,  0.3120, -0.1466])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化


In [ ]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  5.0611, -8.8071, -0.0000],
        [ 6.1681,  0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [ ]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  6.0611, -7.8071,  1.0000])

In [ ]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [ ]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [ ]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [ ]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(9.3132e-10, grad_fn=<MeanBackward0>)

In [ ]:
# 定义层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [ ]:
#实例化层
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.3836,  1.2461, -3.5093],
        [ 0.7237, -0.4288,  1.4457],
        [-0.3805,  0.5559,  0.9742],
        [-0.5626,  0.3044, -0.4748],
        [ 2.4273,  0.3879,  1.3487]], requires_grad=True)

In [ ]:
linear(torch.rand(2, 5))

tensor([[2.8827, 1.5609, 0.2591],
        [2.9475, 1.5306, 0.0000]])

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [ ]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [ ]:
y = torch.zeros(4)
torch.save([x,y],'x-files')
x2,y2 = torch.load('x-files')
(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [ ]:
mydict = {'x':x,'y':y}
torch.save(mydict,'mydict')
mydict2 = torch.load('mydict')
mydict2


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [ ]:
#保存整个模型的参数
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20,256)
    self.output = nn.Linear(256,10)
  
  def forward(self,x):
    return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size = (2,20))
Y = net(X)

In [ ]:
torch.save(net.state_dict(),'mlp.params')  # remember that its functionnal

In [ ]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [6]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [7]:
torch.cuda.device_count()

1

In [8]:
#这里是用来帮助定义类型使用gpu 的函数
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): 
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [9]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [10]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [16]:
Y = torch.rand(2, 3, device=try_gpu())    #这里不要给gpu放到其他的装备上，要记住做运算的时候是同一个gpu
Y

tensor([[0.8232, 0.9670, 0.7847],
        [0.8799, 0.1111, 0.8285]], device='cuda:0')

In [12]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [17]:
Y + Z  #两个是在同一个gpu的情况下进行的运算

tensor([[1.8232, 1.9670, 1.7847],
        [1.8799, 1.1111, 1.8285]], device='cuda:0')

In [20]:
Z.cuda(0) is Z

True

In [21]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [22]:
net(X)

tensor([[-0.1810],
        [-0.1810]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [23]:
net[0].weight.data.device

device(type='cuda', index=0)